In [ ]:
# !pip install minerl

In [ ]:
# from google.colab import drive
import minerl
import gym
import cv2
import numpy as np

import collections
import matplotlib.pyplot as plt
import pickle
import json

import numpy as np

import pickle

#can either set the download path this way or via the OS system variables
# import os
# os.environ['MINERL_DATA_ROOT']="C:\\Users\\danie\\Documents\\Uni\\Masters-Dissertation\\data"

In [ ]:
import sys
sys.path.append("./functions")

from DataHelper import extract_data, parse_minerl_data, get_unique_actions,get_unique_place_actions
from Utils import save_data, load_data
from ActionCombos import get_unique_angles, get_all_action_combos

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
environment = "Treechop"
resources_folder = 'training-resources'

In [ ]:
#downloads data from MineRL API
!python -m minerl.data.download "MineRLTreechop-v0"

In [ ]:
#extracts data from the MineRL API
data = extract_data('MineRL{0}-v0'.format(environment))

In [ ]:
#cleans up the data and puts it in a JSON format
parsed = parse_minerl_data(data)

In [ ]:
#get all unique actions from dataset
unique_actions = get_unique_actions(data[0]['action'])

In [ ]:
#get all unique camera angles from dataset
unique_angles = get_unique_angles(parsed_sequences)

In [ ]:
#only valid for navigate
unique_placements = get_unique_place_actions(parsed_sequences)
save_data("../{0}/unique_placements_{0}.sav".format(resources_folder,environment.lower()),unique_placements)

In [ ]:
#get all unique action combos from datatset. pass in unique_placements if the environment
#data has placements
combos = get_all_action_combos(unique_actions,unique_angles)

In [ ]:
#save all data
save_data("../{0}/parsed_data_{1}.sav".format(resources_folder, environment.lower()), parsed_sequences)
save_data("../{0}/unique_actions_{1}.sav".format(resources_folder,environment.lower()),unique_actions_dataset)
save_data("../{0}/unique_angles_{0}.sav".format(resources_folder,environment.lower()),unique_angles)
save_data("../{0}/action_combos_{1}.sav".format(resources_folder, environment.lower()), combos)